<font color = green >

## Sample 3: language model using keras

</font>

Word level language model is implemented as LSTM for poem of T.Shevchenko


In [1]:
# from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io
# import PyPDF2

In [2]:
import re
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
!wget "https://www.dropbox.com/scl/fi/91769d9iasvrkhfomlj24/Shevchenko_T._Kobzar.txt?rlkey=fw3m5bs1sysywc2mr4j3j0ii3&dl=0" -O /content/sample_data/Shevchenko_T._Kobzar.txt


--2024-02-27 15:23:42--  https://www.dropbox.com/scl/fi/91769d9iasvrkhfomlj24/Shevchenko_T._Kobzar.txt?rlkey=fw3m5bs1sysywc2mr4j3j0ii3&dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9ab1da142bb95a46f1ac7dede0.dl.dropboxusercontent.com/cd/0/inline/COH4DHb-FuCPkikTbWB2tmkMdmi-rzrUE9o21YNVx8WGB9C_SqYV_3h_feDkzV4Sf1ZPEZas4XEtqv92XIZ3_KF5UrGXj9SA3Dd3_ODRbvPuU4kMMLzZU9zl4ms6ZiL_wQ3m6Jsrs-fb0joEjfk5ehEv/file# [following]
--2024-02-27 15:23:43--  https://uc9ab1da142bb95a46f1ac7dede0.dl.dropboxusercontent.com/cd/0/inline/COH4DHb-FuCPkikTbWB2tmkMdmi-rzrUE9o21YNVx8WGB9C_SqYV_3h_feDkzV4Sf1ZPEZas4XEtqv92XIZ3_KF5UrGXj9SA3Dd3_ODRbvPuU4kMMLzZU9zl4ms6ZiL_wQ3m6Jsrs-fb0joEjfk5ehEv/file
Resolving uc9ab1da142bb95a46f1ac7dede0.dl.dropboxusercontent.com (uc9ab1da142bb95a46f1ac7dede0.dl.dropboxusercontent.com)... 162

In [26]:
fn = '/content/sample_data/Shevchenko_T._Kobzar.txt'
with open(fn) as f:
    text = f.read().lower()
# print('corpus length = {","}', len(text))
text = re.sub(r'\[.*\]', "", text)
text = re.sub(r'\d+', "", text)
text = re.sub(r'«|»', "", text)




print ('Len of original text= {:,}'.format (len(text)))
keep = 0.2
text =  text[: int (len(text)* keep)]
print ('Len of snippet= {:,}'.format (len(text)))

print (text[:5000])





Len of original text= 507,759
Len of snippet= 101,551
причинна 
реве та стогне дніпр широкий, сердитий вітер завива, додолу верби гне високі, горами хвилю підійма.
і блідний місяць на ту пору із хмари де-де виглядав, неначе човен в синім морі, то виринав, то потопав. ще треті півні не співали, ніхто нігде не гомонів, сичі в гаю перекликались, та ясен раз у раз скрипів.
в таку добу під горою, біля того гаю,
що чорніє над водою, щось біле блукає.
може, вийшла русалонька матері шукати,
а може, жде козаченька, щоб залоскотати.
не русалонька блукає: то дівчина ходить,
й сама не зна (бо причинна), що такеє робить.
так ворожка поробила,
щоб менше скучала,
щоб, бач, ходя опівночі, спала й виглядала козаченька молодого, що торік покинув. обіщався вернутися,
та, мабуть, і згинув!
не китайкою  покрились козацькії очі,
не вимили біле личко слізоньки дівочі:
орел вийняв карі очі
на чужому полі,
біле тіло вовки з'їли,— така його доля.
дарма щоніч дівчинонька його виглядає.
не вернеться 

In [7]:
vectorizer = CountVectorizer(token_pattern=r'(?u)(?:\b\w+\b|\.|\,|\!|\?|\-|\n)').fit([text])
max_features= 10000

In [8]:
print ('len of features = {:,}\n'.format(len(vectorizer.get_feature_names_out())))
vocab = vectorizer.get_feature_names_out()
print (vocab[:100])


len of features = 4,587

['\n' '!' ',' '-' '.' '?' 'ii' 'iii' 'iv' 'my' 'narodowi' 'nie' 'polska'
 'pozwalam' 'v' 'zginela' 'zyjemy' 'а' 'аби' 'абичию' 'або' 'аж' 'аи'
 'альта' 'альту' 'андреевичу' 'анове' 'ану' 'ані' 'анітелень' 'апреля'
 'архістратига' 'б' 'ба' 'баба' 'бабу' 'бабусенько' 'бабусю' 'багатии'
 'багато' 'багатого' 'багатому' 'багаті' 'базар' 'базари' 'базару'
 'базарі' 'базиліан' 'баи' 'балака' 'бандуристе' 'барвінком' 'барився'
 'барись' 'бариші' 'батечку' 'батогами' 'батька' 'батьки' 'батько'
 'батьком' 'батьку' 'батьків' 'батьківщина' 'бач' 'бачать' 'бачив'
 'бачила' 'бачили' 'бачите' 'бачить' 'бачиш' 'бачся' 'бачте' 'бачу'
 'бевзь' 'без' 'безбатченком' 'безголов' 'безкрає' 'безкраі' 'безславному'
 'безумную' 'белькотали' 'бенкет' 'бенкетували' 'бенкетують' 'бенкетує'
 'бенкеті' 'бере' 'берлин' 'берлині' 'беріть' 'бестія' 'би' 'бивсь'
 'бився' 'бии' 'била' 'били']


In [9]:
word2index = vectorizer.vocabulary_
# word2index['UNK'] = len(word2index)
word2index

{'причинна': 3108,
 '\n': 0,
 'реве': 3241,
 'та': 3870,
 'стогне': 3765,
 'дніпр': 909,
 'широкии': 4393,
 ',': 2,
 'сердитии': 3478,
 'вітер': 597,
 'завива': 1203,
 'додолу': 942,
 'верби': 300,
 'гне': 669,
 'високі': 420,
 'горами': 736,
 'хвилю': 4144,
 'підіи': 3203,
 'ма': 1977,
 '.': 4,
 'і': 4549,
 'бліднии': 125,
 'місяць': 2180,
 'на': 2185,
 'ту': 3994,
 'пору': 2958,
 'із': 4563,
 'хмари': 4165,
 'де': 859,
 '-': 3,
 'виглядав': 355,
 'неначе': 2320,
 'човен': 4302,
 'в': 240,
 'синім': 3517,
 'морі': 2130,
 'то': 3946,
 'виринав': 401,
 'потопав': 3000,
 'ще': 4453,
 'треті': 3977,
 'півні': 3185,
 'не': 2275,
 'співали': 3695,
 'ніхто': 2408,
 'нігде': 2397,
 'гомонів': 727,
 'сичі': 3531,
 'гаю': 636,
 'перекликались': 2663,
 'ясен': 4522,
 'раз': 3226,
 'у': 4022,
 'скрипів': 3549,
 'таку': 3881,
 'добу': 925,
 'під': 3189,
 'горою': 745,
 'біля': 237,
 'того': 3952,
 'що': 4466,
 'чорніє': 4329,
 'над': 2214,
 'водою': 473,
 'щось': 4473,
 'біле': 232,
 'блукає': 124

In [10]:
# import operator

In [11]:
index2word = {v:k for k,v in word2index.items()}
index2word

{3108: 'причинна',
 0: '\n',
 3241: 'реве',
 3870: 'та',
 3765: 'стогне',
 909: 'дніпр',
 4393: 'широкии',
 2: ',',
 3478: 'сердитии',
 597: 'вітер',
 1203: 'завива',
 942: 'додолу',
 300: 'верби',
 669: 'гне',
 420: 'високі',
 736: 'горами',
 4144: 'хвилю',
 3203: 'підіи',
 1977: 'ма',
 4: '.',
 4549: 'і',
 125: 'бліднии',
 2180: 'місяць',
 2185: 'на',
 3994: 'ту',
 2958: 'пору',
 4563: 'із',
 4165: 'хмари',
 859: 'де',
 3: '-',
 355: 'виглядав',
 2320: 'неначе',
 4302: 'човен',
 240: 'в',
 3517: 'синім',
 2130: 'морі',
 3946: 'то',
 401: 'виринав',
 3000: 'потопав',
 4453: 'ще',
 3977: 'треті',
 3185: 'півні',
 2275: 'не',
 3695: 'співали',
 2408: 'ніхто',
 2397: 'нігде',
 727: 'гомонів',
 3531: 'сичі',
 636: 'гаю',
 2663: 'перекликались',
 4522: 'ясен',
 3226: 'раз',
 4022: 'у',
 3549: 'скрипів',
 3881: 'таку',
 925: 'добу',
 3189: 'під',
 745: 'горою',
 237: 'біля',
 3952: 'того',
 4466: 'що',
 4329: 'чорніє',
 2214: 'над',
 473: 'водою',
 4473: 'щось',
 232: 'біле',
 124: 'блукає'

In [12]:
word_tokenizer = vectorizer.build_tokenizer()
raw_tokens = word_tokenizer(text)

tokens = sorted(set(raw_tokens))
print('len of all tokens = {:,}'.format(len(raw_tokens)))
print('len of unique tokens = {:,}'.format(len(tokens)))

len of all tokens = 26,094
len of unique tokens = 4,587


In [13]:
n_context  = 7
step = 1 # shift to build new sample
contexts = []
targets = []
for i in range(0, len(raw_tokens) - n_context, step):
    contexts.append(raw_tokens[i: i + n_context])
    targets.append(raw_tokens[i + n_context])
print('len(samples) = {:,}'.format(len(contexts)))

for i in range (20):
    print ('{} -> {}'.format (contexts[i], targets[i]))


len(samples) = 26,087
['причинна', '\n', 'реве', 'та', 'стогне', 'дніпр', 'широкии'] -> ,
['\n', 'реве', 'та', 'стогне', 'дніпр', 'широкии', ','] -> сердитии
['реве', 'та', 'стогне', 'дніпр', 'широкии', ',', 'сердитии'] -> вітер
['та', 'стогне', 'дніпр', 'широкии', ',', 'сердитии', 'вітер'] -> завива
['стогне', 'дніпр', 'широкии', ',', 'сердитии', 'вітер', 'завива'] -> ,
['дніпр', 'широкии', ',', 'сердитии', 'вітер', 'завива', ','] -> додолу
['широкии', ',', 'сердитии', 'вітер', 'завива', ',', 'додолу'] -> верби
[',', 'сердитии', 'вітер', 'завива', ',', 'додолу', 'верби'] -> гне
['сердитии', 'вітер', 'завива', ',', 'додолу', 'верби', 'гне'] -> високі
['вітер', 'завива', ',', 'додолу', 'верби', 'гне', 'високі'] -> ,
['завива', ',', 'додолу', 'верби', 'гне', 'високі', ','] -> горами
[',', 'додолу', 'верби', 'гне', 'високі', ',', 'горами'] -> хвилю
['додолу', 'верби', 'гне', 'високі', ',', 'горами', 'хвилю'] -> підіи
['верби', 'гне', 'високі', ',', 'горами', 'хвилю', 'підіи'] -> ма
['гне'

In [14]:
import numpy as np

In [19]:
print('Converting to one-hot vectors...')
x = np.zeros((len(contexts), n_context, len(tokens)), dtype=bool)
y = np.zeros((len(contexts), len(tokens)), dtype=bool)
for i, context in enumerate(contexts):
    for t, token in enumerate(context):
        x[i, t, word2index[token]] = 1
    y[i, word2index[targets[i]]] = 1
print('Done.')

Converting to one-hot vectors...
Done.


In [20]:
print('Build model (single LSTM)...')
model = Sequential()
model.add(LSTM(128, input_shape=(n_context, len(tokens))))
model.add(Dense(len(tokens), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

def sample(preds, diversity=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / diversity
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print('\n----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(raw_tokens) - n_context - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('--- diversity:', diversity)
        context = raw_tokens[start_index: start_index + n_context]
        print('--- Generated with the following context: {}'.format(context))
        # sys.stdout.write(generated)
        orig_context = list (context)
        generated = []
        for i in range(40): # number of tokens to gerenerate
            x_pred = np.zeros((1, n_context, len(tokens)))
            for t, token in enumerate(context):
                x_pred[0, t, word2index[token]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = index2word[next_index]

            generated += [next_word]

            # update context for next pass
            context = context[1:] + [next_word]

        print ('{} -> {}'.format(' '.join(orig_context) , ' '.join(generated)))
            #             sys.stdout.write(next_char)
            #             sys.stdout.flush()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Build model (single LSTM)...


Epoch 1/60
204/204 [==============================] - ETA: 0s - loss: 6.3161
----- Generating text after Epoch: 0
--- diversity: 0.2
--- Generated with the following context: [',', '\n', 'щоб', 'не', 'побачили', ',', 'бач']
, 
 щоб не побачили , бач -> 
 , 
 , , , , , , , 
 , 
 , . , , , , . , , , , , , , . , , , , 
 
 , 
 , , , .
--- diversity: 0.5
--- Generated with the following context: [',', '\n', 'щоб', 'не', 'побачили', ',', 'бач']
, 
 щоб не побачили , бач -> 
 не . 
 , , 
 , , 
 
 не , , 
 . ! , що , . , 
 . . , 
 
 . 
 
 ! , , . , , . не 

--- diversity: 1.0
--- Generated with the following context: [',', '\n', 'щоб', 'не', 'побачили', ',', 'бач']
, 
 щоб не побачили , бач -> - . ? в пішла . аж , щоб яже дожидали ні і 
 . , . ну не , муки лихо і правда , та теся . , раз а , як . по я мертва близько доле от
--- diversity: 1.2
--- Generated with the following context: [',', '\n', 'щоб', 'не', 'побачили', ',', 'бач']
, 
 щоб не побачили , бач -> бенкетують 
 другіи загуло вози ж

KeyboardInterrupt: 

**Output after 60 epochs:**
```
Epoch 60/60
26087/26087 [==============================] - 46s 2ms/step - loss: 2.0366

----- Generating text after Epoch: 59
--- diversity: 0.2
--- Generated with the following context: ['то', 'и', 'стара', 'мати', ',', '\n', 'що']
то и стара мати ,
 що -> жить на !
 з украі ну на міи ,

 тяжко лихо немає .
 а хто долі серед без , як пішов ? . .
 то ж ж , ж ти ж ж , і ,
--- diversity: 0.5
--- Generated with the following context: ['то', 'и', 'стара', 'мати', ',', '\n', 'що']
то и стара мати ,
 що -> розумні на , а тим часом
 ярема не ножі , з боже є з козаками ? думи моі , крові . іди , де мені з нащо ? де будеш , ось ?
 не слова ,
 що
--- diversity: 1.0
--- Generated with the following context: ['то', 'и', 'стара', 'мати', ',', '\n', 'що']
то и стара мати ,
 що -> привела на чужині зозуленька . !
 не пожари воля знали пии темнии , сеи згадаєш сизии один вздовж бабусю мовчіть вии запорожці , прилини втирають хмари жупани полюбила поглядає єсть золоті кличе лежить треті була де ще степ цілує
--- diversity: 1.2
--- Generated with the following context: ['то', 'и', 'стара', 'мати', ',', '\n', 'що']
то и стара мати ,
 що -> навіки щиро хали щиро греблю поганці , московщину кохаи знала , весна панове ревуть . літа тихесенько
 плаче козацького таке стоя
 москаля , сховається гонта
 крові чорноброві ходімо побачиш цілує уже журись
 слухать чує неба з
<keras.callbacks.callbacks.History at 0x1a8b797190>
```

<font color = green >

## Sample 2A: language model using keras

</font>

Character level language model is implemented as LSTM for poem of T.Shevchenko


In [22]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text into sequences of maxlen characters
maxlen = 40
step = 3 # shift to build new sample
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('len(samples) = {:,}'.format(len(sentences)))


print('Converting to one-hot vectors...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


print('Build model (single LSTM)...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print('\n----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generated: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400): # number of chars to gerenerate
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

total chars: 69
len(samples) = 33,837
Converting to one-hot vectors...
Build model (single LSTM)...


Epoch 1/60
265/265 [==============================] - ETA: 0s - loss: 3.2873
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generated: " може,
якби малось сили;
заспівав би,— б"
 може,
якби малось сили;
заспівав би,— би о ота оо аа а та  ааи на а та а  а са и да  ио и о ии и а ва и а  оо а а о аи та и и аа ни ни на ии та  па иа  аа ва на а  оаи  та  та  оои я ла аа та ии о оя ла и  а а аи па аи и па а и о а ри и  аа а та, ва а оа ио о аи и на  аона и наи  о аа а , та а по о аи, т та а а ваи на я ооа до оа та  аа ва ии  ооа и та и а ма паи и оо аа  аа ти  а аиа а ва а а а а и а а еи о а ии а а , и за и а та по а----- diversity: 0.5
----- Generated: " може,
якби малось сили;
заспівав би,— б"
 може,
якби малось сили;
заспівав би,— баоеа таа оӑи, пиасна та  ялсе
с! ако уои йо
вап ии?аи ва  ооти  чаи , до оввя , виаовр яи таа а аеи не нанан , уа,  

KeyboardInterrupt: 

Generating text after Epoch: 59 (diversity: 0.5)
<br>
`як він, свої думи тії
і серце убо"
оре.
як він, свої думи тії
і серце убогії горили.
а до то прості на собі, поки не знать... а не чуть, а там діти?
і могили погину, синіїм піде з киїти, а як і на всеї дожить.
а ще сумуючи, поки все бордная! привела в світі та зарегожа поможем засталося. не просла сусіди, а що криниці, молоденька веселі,
ми за то розказила
і при другаять. а може, співає,
і все знає,
і дівила світі, як то плети,
а я ж думать за що, нехай була одно`

Generating text after Epoch: 59 (diversity: 1.0)
<br>
`а галайда, знай, гукає:
«кари ляхам, спьшає німили: сходиться знали
і все роззуттва сором ми по всі жали, і ти...» — коли їла.
мов ні його співали? кричіє! роз кету, скажи мате,
що й золобе бога, чую!
за тій світу заборкає.
з землих я княшіво-ховав,
у хату підерячки, щоб великпертика й не веселі, плачуть передову. .....
нуме понадвала стане, тому так! ея
і деревамі,
а точки.
я сказать, у бав. ! тобі їй пузандій молоденько і`


# Hometask

1) Find text to train (any book)<br>
2) Build train and validation set <br>
3) Train bidirectional language model that predicts the POS of word being based on its `n_context= 3` neighbours from the left and `n_context= 3` neighbours from the right <br>
4) Evaluate the model